print(1+2)


# **Main Code**

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from colorama import init, Fore, Style

init(autoreset=True)

# === SETTINGS ===
#if taking the input through webcam internal or external(try 0 or 1)
#INPUT_SOURCE = 0
INPUT_SOURCE = r"C:\Users\Chris Dias\Desktop\Chris_bca\Crowdsense_detection\Overcrowded\CRAZY STAMPEDE AT INDIAN TRAIN STATION.mp4"
OUTPUT_FOLDER = r"C:\Users\Chris Dias\Desktop\Chris_bca\Crowdsense_detection\Outputs"
OUTPUT_FILENAME = "filled_heatmap_result.mp4"

# --- 1. SETUP PATHS ---
OUTPUT_FULL_PATH = os.path.join(OUTPUT_FOLDER, OUTPUT_FILENAME)
if not os.path.exists(OUTPUT_FOLDER): os.makedirs(OUTPUT_FOLDER)

if not os.path.exists(INPUT_SOURCE):
    print(Fore.RED + f"CRITICAL ERROR: File not found: {INPUT_SOURCE}")
    exit()

print(Fore.CYAN + "Loading YOLOv11 Model...")
model = YOLO('yolo11n.pt') 

# --- 2. SETUP VIDEO ---
cap = cv2.VideoCapture(INPUT_SOURCE, cv2.CAP_FFMPEG)
if not cap.isOpened():
    print(Fore.RED + "ERROR: Could not open video.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_FULL_PATH, fourcc, fps, (width, height))

print(Fore.CYAN + f"Processing Transparent Mode: {width}x{height}")

while True:
    ret, frame = cap.read()
    if not ret: break

    # === AGGRESSIVE SETTINGS (To match the "300" count) ===
    results = model.predict(frame, conf=0.10, iou=0.90, imgsz=640, classes=[0], verbose=False)

    total_persons = 0
    
    # Create a separate layer for drawing the filled boxes
    overlay = frame.copy()
    
    # 1. DRAW FILLED BOXES
    if results[0].boxes:
        for box in results[0].boxes:
            total_persons += 1
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # --- COLOR LOGIC (Hackathon Tuned) ---
            box_color = (0, 255, 0) # Green (Safe)
            
            if total_persons > 20: 
                box_color = (0, 165, 255) # Orange (Warning)
            
            # TRIGGER HAPPY: Turns Red at 25 people now
            if total_persons > 25: 
                box_color = (0, 0, 255)   # Red (Critical)

            # Draw FILLED rectangle
            cv2.rectangle(overlay, (x1, y1), (x2, y2), box_color, -1)

    # 2. BLEND OVERLAY (Transparency)
    cv2.addWeighted(overlay, 0.4, frame, 0.6, 0, frame)

    # 3. DASHBOARD LOGIC
    status = "Normal"
    status_color = (0, 255, 0) # Green text

    # Matches the Box Color Logic above
    if total_persons > 25:
        status = "CRITICAL RISK"
        status_color = (0, 0, 255) # Red text
    elif total_persons > 20:
        status = "High Density"
        status_color = (0, 165, 255) # Orange text

    # --- DRAW INTERFACE ---
    # 1. White Background Bar (Slightly taller for better visibility)
    cv2.rectangle(frame, (0, 0), (width, 90), (255, 255, 255), -1)
    
    # 2. "Stampede Chance" Text (Left Side)
    cv2.putText(frame, f"Stampede Chance: {status}", (20, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)
    
    # 3. "Count" Text (Right Side)
    cv2.putText(frame, f"Count: {total_persons}", (width - 250, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

    # Write & Show
    out.write(frame)
    cv2.imshow('Transparent Heatmap', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print(Fore.GREEN + f"Saved Transparent video to: {OUTPUT_FULL_PATH}")

Loading YOLOv11 Model...
Processing Transparent Mode: 630x360
Saved Transparent video to: C:\Users\Chris Dias\Desktop\Chris_bca\Crowdsense_detection\Outputs\filled_heatmap_result.mp4


# **To Check For camera**

In [ ]:
import cv2

def test_camera(index):
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        print(f"Your Camera {index} is NOT available.")
        return
    
    print(f"Your Camera {index} is working! Press 'q' to close it.")
    
    while True:
        ret, frame = cap.read()
        if not ret: break
        cv2.imshow(f'Testing Camera {index}', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# --- CHANGE THIS NUMBER TO TEST ---
test_camera(0) 
# test_camera(1) 